<a href="https://colab.research.google.com/github/rishabh135/2015/blob/master/LlamaIndex_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LlamaIndex: Connecting Multiple Data Sources to Your LLMs

In this notebook, we'll be going through some examples of what makes LlamaIndex a great library when you need to attach multiple disparate sources of data to your LLMs!

While we'll be using OpenAI's `GPT-4` in this demonstration, you could extend this to many different open-source LLMs, providing you were hosting them!

Let's get started, as always, by grabbing our first dependency.

### BOILERPLATE

In [ ]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Dependencies and LlamaIndex Setup

#### OpenAI LLM Setup

We're going to need to do a bit of work to set up our OpenAI LLM the way we want!

In [ ]:
!pip install -U -q llama-index openai

By default, LLamaIndex will use `text-davinci-003`, but we'll want to go ahead and modify that - which is a straightforward process.

First, let's make sure our OpenAI API key is visibile in our environment.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR OPENAI API KEY HERE"

Now we can go ahead and change the underlying LLM that powers our LlamaIndex!

You'll notice we're using the `chunk_size` parameter to define the size of our split data. You can play with this value to try larger/smaller chunks and see how they impact performance!

In [ ]:
from llama_index import ServiceContext
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index.llms import OpenAI

# general configurations
chunk_size = 2048
llm = OpenAI(temperature=0, model="gpt-4", streaming=True)
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=chunk_size)

# configurations for vector store
text_splitter = TokenTextSplitter(chunk_size=chunk_size)
node_parser = SimpleNodeParser(text_splitter=text_splitter)

#### Pinecone Setup

Here we'll following the Pinecone [quickstart guide](https://docs.pinecone.io/docs/quickstart) to grab ourselves a Pinecone API so we can leverage it as our vector store.

You could substitute your favourite vector store here with no issues!

In [ ]:
!pip install -U -q pinecone-client transformers

In [ ]:
import pinecone
import os

api_key = "YOUR PINECONE API KEY HERE"
pinecone.init(api_key=api_key, environment="us-west1-gcp-free")

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")

In [ ]:
pinecone_index = pinecone.Index("quickstart")

Now we can set up our LLamaIndex `PineconeVectorStore`!

In [ ]:
from llama_index.vector_stores import PineconeVectorStore
from llama_index.storage import StorageContext
from llama_index import VectorStoreIndex

# connect and name the PineconeVectorStore
team_vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index, namespace="wikipedia_team_info"
)

# allow the PineconeVectorStore to be used as storage
storage_context = StorageContext.from_defaults(vector_store=team_vector_store)

# allow the creation of an Index
team_vector_index = VectorStoreIndex([], storage_context=storage_context)

### Data Collection

We're going to look at NHL hockey data - so let's start grabbing some!

We'll start with some `.csv` files that contains season data all the way back to the 2008-2009 season!

In [ ]:
# team statistics
!wget https://moneypuck.com/moneypuck/playerData/careers/gameByGame/all_teams.csv

--2023-08-03 01:23:55--  https://moneypuck.com/moneypuck/playerData/careers/gameByGame/all_teams.csv
Resolving moneypuck.com (moneypuck.com)... 104.21.93.205, 172.67.214.96, 2606:4700:3032::6815:5dcd, ...
Connecting to moneypuck.com (moneypuck.com)|104.21.93.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘all_teams.csv.1’

all_teams.csv.1         [           <=>      ]  98.81M  10.6MB/s    in 9.4s    

2023-08-03 01:24:05 (10.5 MB/s) - ‘all_teams.csv.1’ saved [103606286]



In [ ]:
# player bio data
!wget https://moneypuck.com/moneypuck/playerData/playerBios/allPlayersLookup.csv

--2023-08-03 01:24:05--  https://moneypuck.com/moneypuck/playerData/playerBios/allPlayersLookup.csv
Resolving moneypuck.com (moneypuck.com)... 104.21.93.205, 172.67.214.96, 2606:4700:3032::6815:5dcd, ...
Connecting to moneypuck.com (moneypuck.com)|104.21.93.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘allPlayersLookup.csv.1’

allPlayersLookup.cs     [ <=>                ] 190.53K  --.-KB/s    in 0.1s    

2023-08-03 01:24:05 (1.31 MB/s) - ‘allPlayersLookup.csv.1’ saved [195102]



Now that we have some `.csv` files, let's go ahead and build an SQL database out of them!

We'll be leveraing `sqlalchemy` to do this!

In [ ]:
!pip install -U -q sqlalchemy

Let's set up some `pd.DataFrames` to collect our `csv` files into a more usable format.

In [ ]:
import pandas as pd

all_teams_df = pd.read_csv("all_teams.csv")

In [ ]:
all_teams_df.head()

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,...,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
0,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,other,...,1.0,1.000,0.0,0.017,0.000,0.000,0.037,0.037,0.037,0
1,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,all,...,31.0,30.369,5.0,0.396,0.168,0.168,2.917,2.833,2.714,0
2,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,5on5,...,20.0,19.369,3.0,0.237,0.168,0.168,1.862,1.777,1.665,0
3,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,4on5,...,9.0,9.000,1.0,0.124,0.000,0.000,0.795,0.795,0.789,0
4,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,5on4,...,1.0,1.000,1.0,0.019,0.000,0.000,0.224,0.224,0.224,0


In [ ]:
all_teams_df = all_teams_df.rename(columns={"team" : "team_initials", "name" : "team_name", "position" : "team_position"})

In [ ]:
all_players_df = pd.read_csv("allPlayersLookup.csv")

In [ ]:
all_players_df = all_players_df.rename(columns={"team" : "player_team", "name" : "player_name", "position" : "player_position"})

In [ ]:
all_players_df.head()

,playerId,player_name,player_position,player_team,birthDate,weight,height,nationality,shootsCatches,primaryNumber,primaryPosition
0,8478421,A.J. Greer,L,BOS,1996-12-14,210.0,"6' 3""",CAN,L,24.0,L
1,8477180,Aaron Dell,G,SJS,1989-05-04,205.0,"6' 0""",CAN,L,30.0,G
2,8465992,Aaron Downey,R,DET,1974-08-27,215.0,"6' 1""",CAN,R,44.0,R
3,8477932,Aaron Ekblad,D,FLA,1996-02-07,220.0,"6' 4""",CAN,R,5.0,D
4,8471451,Aaron Gagnon,C,WPG,1986-04-24,186.0,"5' 11""",CAN,R,21.0,C


Now that we have those dataframes, let's create some SQL Tables!

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", future=True)

In [ ]:
all_teams_df.to_sql('all_team_data', engine)

190300

In [ ]:
all_players_df.to_sql('all_player_data', engine)

3034

Let's verify our process worked!

In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
  result = conn.execute(text("SELECT player_name, player_position, player_team FROM all_player_data WHERE player_name is 'Aaron Dell'"))

  for row in result:
    print(row)

('Aaron Dell', 'G', 'SJS')


Great! Now that we've done that - let's add some context by grabbing some information about the teams from Wikipedia!

In [ ]:
!pip install -U -q wikipedia

In [ ]:
# grab the teams in the `all_team_data` database
with engine.connect() as conn:
  results = conn.execute(text("SELECT DISTINCT team_name FROM all_team_data"))
  team_list = [result.team_name for result in results]

print(team_list)

['NYR', 'SJS', 'PIT', 'EDM', 'WPG', 'PHI', 'DAL', 'N.J', 'T.B', 'MIN', 'COL', 'ANA', 'VGK', 'TOR', 'NYI', 'MTL', 'SEA', 'STL', 'FLA', 'BUF', 'LAK', 'ARI', 'OTT', 'WSH', 'NJD', 'TBL', 'CBJ', 'CHI', 'NSH', 'CGY', 'BOS', 'DET', 'VAN', 'CAR', 'S.J', 'ATL', 'L.A']


In [ ]:
assert len(team_list) == 33

AssertionError: ignored

Uh-oh! There seems to be duplicate team entries in the databse - let's build a map between those teams.

Also, for the curious, this data goes back to 2008 which means that it has the current 32 NHL teams, as well as the now Winnipeg Jets - the Atlanta Thrashers.

In [ ]:
for team in team_list:
  print(team)

NYR
SJS
PIT
EDM
WPG
PHI
DAL
N.J
T.B
MIN
COL
ANA
VGK
TOR
NYI
MTL
SEA
STL
FLA
BUF
LAK
ARI
OTT
WSH
NJD
TBL
CBJ
CHI
NSH
CGY
BOS
DET
VAN
CAR
S.J
ATL
L.A


As we can see, the duplication appears to be related to the teams like the LA Kings, which appear both as `LAK` and `L.A`.

Let's disambiguate those and build a new list.

In [ ]:
team_list = [team for team in team_list if "." not in team]

In [ ]:
assert len(team_list) == 33

Let's disambiguate the initialisms to team names to make it easier to collect Wikipedia information.

In [ ]:
team_str = """
*AFM – [[Atlanta Flames]]
*ANA – Mighty Ducks of Anaheim/[[Anaheim Ducks|Anaheim Ducks]]
*ARI – [[Arizona Coyotes]]
*ATL – [[Atlanta Thrashers]]
*BOS – [[Boston Bruins]]
*BRK – [[New York Americans|Brooklyn Americans]]
*BUF – [[Buffalo Sabres]]
*CAR – [[Carolina Hurricanes]]
*CBJ – [[Columbus Blue Jackets]]
*CGS – Bay Area Seals/[[California Golden Seals|California Golden Seals]]
*CGY – [[Calgary Flames]]
*CHI – Chicago Black Hawks/[[Chicago Blackhawks|Blackhawks]]
*CLE – [[Cleveland Barons (NHL)|Cleveland Barons]]
*CLR – [[Colorado Rockies (NHL)|Colorado Rockies]]
*COL – [[Colorado Avalanche]]
*DAL – [[Dallas Stars]]
*DCG – [[Detroit Cougars (NHL)|Detroit Cougars]]
*DET – [[Detroit Red Wings]]
*DFL – [[Detroit Falcons (NHL)|Detroit Falcons]]
*EDM – [[Edmonton Oilers]]
*FLA – [[Florida Panthers]]
*HAM – [[Hamilton Tigers (ice hockey)|Hamilton Tigers]]
*HFD – [[Hartford Whalers]]
*KCS – [[Kansas City Scouts]]
*LAK – [[Los Angeles Kings]]
*MIN – [[Minnesota Wild]]
*MMR – [[Montreal Maroons]]
*MNS – [[Minnesota North Stars]]
*MTL – [[Montreal Canadiens]]
*MWN – [[Montreal Wanderers]]
*NJD – [[New Jersey Devils]]
*NSH – [[Nashville Predators]]
*NYA – [[New York Americans]]
*NYI – [[New York Islanders]]
*NYR – [[New York Rangers]]
*OAK – California/[[Oakland Seals|Oakland Seals]]
*OTT – [[Ottawa Senators]]
*PHI – [[Philadelphia Flyers]]
*PHX – [[Phoenix Coyotes]]
*PIR – [[Pittsburgh Pirates (NHL)|Pittsburgh Pirates]]
*PIT – [[Pittsburgh Penguins]]
*QBD – [[Quebec Bulldogs]]
*QUA – [[Philadelphia Quakers (NHL)|Philadelphia Quakers]]
*QUE – [[Quebec Nordiques]]
*SEA – [[Seattle Kraken]]
*SEN – [[Ottawa Senators (original)]]
*SLE – [[St. Louis Eagles]]
*SJS – [[San Jose Sharks]]
*STL – [[St. Louis Blues]]
*TAN – Toronto Hockey Club/[[Toronto Arenas|Toronto Arenas]]
*TBL – [[Tampa Bay Lightning]]
*TOR – [[Toronto Maple Leafs]]
*TSP – [[Toronto St. Patricks]]
*VAN – [[Vancouver Canucks]]
*VGK – [[Vegas Golden Knights]]
*WIN – [[Winnipeg Jets (1972–96)|Winnipeg Jets (original)]]
*WPG – [[Winnipeg Jets]]
*WSH – [[Washington Capitals]]
"""

team_str_list = team_str.splitlines()

In [ ]:
from collections import defaultdict

team_map = defaultdict(str)

for team in team_str_list:
  if len(team) < 2:
    continue
  if len(team.split("–")) > 2:
    continue
  initialism, name = team.split("–")
  team_map[initialism.replace("*", "").strip()] = name.replace("[", "").replace("]", "").strip()


Alright, now we can start collecting context for our appliction!

In [ ]:
from llama_index.readers.wikipedia import WikipediaReader

team_name_list = [team_map[team].split("/")[0] for team in team_list]

wiki_docs = WikipediaReader().load_data(pages=team_name_list, auto_suggest=False)

### Index Creation

Now that we have data sources, let's get to creating some Indexes so we can intelligently access them to perform our desired tasks.

#### SQL Index

We'll start with the `SQLDatabase` index.

First up, let's put our database into a LlamaIndex friendly format and point at our tables/engine.

In [ ]:
from llama_index import SQLDatabase, SQLStructStoreIndex

sql_database = SQLDatabase(engine, include_tables=["all_team_data", "all_player_data"])

Now we can set up our `NLSQLTableQueryEngine`.

This does what it says on the tin:

Converts natural language queries to SQL queries and queries against the database.

In [ ]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["all_team_data", "all_player_data"]
)

#### Vector Index

Now we can set up our traditional vector index!

In [ ]:
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
for team, wiki_doc in zip(team_list, wiki_docs):
  nodes = node_parser.get_nodes_from_documents([wiki_doc])
  for node in nodes:
    node.metadata = {"title" : team_map[team]}
  team_vector_index.insert_nodes(nodes)

Upserted vectors:   0%|          | 0/7 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/6 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/8 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/8 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/4 [00:00<?, ?it/s]

...

Upserted vectors:   0%|          | 0/6 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/6 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/9 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/6 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/4 [00:00<?, ?it/s]

### Set up Query Engines

Query Engines are at the heart of what makes LlamaIndex tick.

Similar to LangChain's chains, Query Engines are used to connect your user's queries to the LLM through the Index!

In [ ]:
from llama_index.query_engine import SQLAutoVectorQueryEngine, RetrieverQueryEngine
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.indices.vector_store import VectorIndexAutoRetriever
from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo

team_vector_store_info = VectorStoreInfo(
    content_info="articles about different NHL teams",
    metadata_info=[
        MetadataInfo(
            name="title",
            type="str",
            description="The name of the NHL team.")
    ]
)

team_vector_auto_retriever = VectorIndexAutoRetriever(
    team_vector_index, vector_store_info=team_vector_store_info
)

team_retriever_query_engine = RetrieverQueryEngine.from_args(
    team_vector_auto_retriever, service_context=service_context
)

In [ ]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into an SQL query over tables containing:"
        "all_player_data, containing biographical information on NHL hockey players."
        "all_team_data, containing stats related to NHL hockey teams"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=team_retriever_query_engine,
    description=f"Useful for answering semantic questions about NHL Hockey Teams"
)

query_engine = SQLAutoVectorQueryEngine(
    sql_tool,
    vector_tool,
    service_context=service_context
)

### Testing it out!

Now we can see how our application performs.

In [ ]:
response = query_engine.query("What team has the most points?")

Querying SQL database: Useful for translating a natural language query into an SQL query over tables containing all_team_data, which contains stats related to NHL hockey teams.
SQL query: SELECT team_name, SUM(xGoalsFor) AS total_points 
FROM all_team_data 
GROUP BY team_name 
ORDER BY total_points DESC;
SQL response:  The team with the most points is the Pittsburgh Penguins with 7933.689000000015 points.
Transformed query given SQL response: None


In [ ]:
print(response)

The team with the most points is the Pittsburgh Penguins with a total of 7933.689 points. Several players contributed to this high score, including notable names such as Mario Lemieux, Jaromir Jagr, Sidney Crosby, Evgeni Malkin, and Marc-Andre Fleury, among others.


Not too shabby!

Let's do a more complex query and see if it can keep up!

In [ ]:
response = query_engine.query("Who is the oldest current player on the Winnipeg Jets?")

Querying SQL database: This choice is useful for translating a natural language query into an SQL query over tables containing biographical information on NHL hockey players.
SQL query: SELECT player_name, birthDate 
FROM all_player_data 
WHERE player_team = 'Winnipeg Jets' 
ORDER BY birthDate ASC 
LIMIT 1;
SQL response:  The oldest current player on the Winnipeg Jets is Mathieu Perreault, who was born on January 5, 1988.
Transformed query given SQL response: None


In [ ]:
print(response)

 The oldest current player on the Winnipeg Jets is Mathieu Perreault, who was born on January 5, 1988.


Let's try one more that combines a few of our tables and wikipedia articles!

In [ ]:
response = query_engine.query("How old was the Winnipeg Jets captain in 2018? Will they be playing with the Jets in 2023?")

Querying SQL database: Useful for translating a natural language query into an SQL query over tables containing biographical information on NHL hockey players.
SQL query: SELECT player_name, birthDate, team_name 
FROM all_player_data 
JOIN all_team_data 
ON all_player_data.player_team = all_team_data.team_initials 
WHERE team_name = 'Winnipeg Jets' 
AND season = 2018 
AND primaryPosition = 'C' 
ORDER BY birthDate ASC;
SQL response: 
In 2018, the Winnipeg Jets captain was Blake Wheeler, who was born on August 31, 1986. It is unclear if he will still be playing with the Jets in 2023.
Transformed query given SQL response: Will Blake Wheeler be playing with the Winnipeg Jets in 2023?
query engine response: The context information does not provide specific details about Blake Wheeler's status with the Winnipeg Jets in 2023.
Final response: In 2018, the Winnipeg Jets captain was Blake Wheeler, who was born on August 31, 1986, making him 32 years old in 2018. As for whether he will still be p

In [ ]:
print(response)

In 2018, the Winnipeg Jets captain was Blake Wheeler, who was born on August 31, 1986, making him 32 years old in 2018. As for whether he will still be playing with the Jets in 2023, the information available does not provide specific details about Blake Wheeler's status with the Winnipeg Jets in 2023.


In [ ]:
response = query_engine.query("What special traditions do the Winnipeg Jets fans have during the playoffs?")

Querying other query engine: Useful for answering semantic questions about NHL Hockey Teams
Query Engine response: The Winnipeg Jets fans have a special tradition called the Winnipeg Whiteout during the playoffs. This tradition dates back to 1987 when fans of the original Jets franchise were asked to wear white clothing at the home playoff games. It was created as a response to the "C of Red" created by fans of the Calgary Flames, whom the Jets were facing in the first round of the 1987 Stanley Cup playoffs. The Jets eliminated the Flames in six games, and fans wore white for every home playoff game thereafter. The Winnipeg Jets resurrected this tradition when they qualified for the 2015 Stanley Cup playoffs.


In [ ]:
print(response)

The Winnipeg Jets fans have a special tradition called the Winnipeg Whiteout during the playoffs. This tradition dates back to 1987 when fans of the original Jets franchise were asked to wear white clothing at the home playoff games. It was created as a response to the "C of Red" created by fans of the Calgary Flames, whom the Jets were facing in the first round of the 1987 Stanley Cup playoffs. The Jets eliminated the Flames in six games, and fans wore white for every home playoff game thereafter. The Winnipeg Jets resurrected this tradition when they qualified for the 2015 Stanley Cup playoffs.


### Conclusion

As you can see, we can leverage LlamaIndex to great effect to query across multiple disparate data source. It can also handle querying across SQL databases, including doing joins!